In [3]:
import os
import random
import torch
import numpy as np
import pickle as pkl
from analysis import *
import argparse
from sys import platform

In [4]:
torch.manual_seed(12)
torch.cuda.manual_seed(12)
np.random.seed(12)
torch.backends.cudnn.deterministics = True
torch.set_num_threads(1)

In [5]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())


print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
NVIDIA TITAN V


In [6]:
# iv: image option
length = 440
channel = 128
min_CNN = 200
n_classes = 40
classes = range(n_classes)


In [7]:
if platform == "linux" or platform == "linux2":
    torch_models_dir = r"/media/titan/AI Research1/Data/CVPR2017"
elif platform == "win32":
    torch_models_dir = r"D:\Data\CVPR2021-02785\CVPR2021-02785\preprocessed\torch_models"
block_splits_all, block_splits_single, eeg_14_70, eeg_55_95, eeg_5_95, eeg_raw = os.listdir(torch_models_dir)
print(os.listdir(torch_models_dir))

['block_splits_by_image_all.pth', 'block_splits_by_image_single.pth', 'eeg_14_70_std.pth', 'eeg_55_95_std.pth', 'eeg_5_95_std.pth', 'eeg_signals_raw_with_mean_std.pth']


In [8]:
eeg_dataset = os.path.join(torch_models_dir, eeg_5_95)
splits_all_path = os.path.join(torch_models_dir, block_splits_all)
splits_single_path = os.path.join(torch_models_dir, block_splits_single)
# splits_path = os.path.join(torch_models_dir, splits_shuffled_path)
print(eeg_dataset,'\n', splits_all_path, '\n', splits_single_path)

/media/titan/AI Research1/Data/CVPR2017/eeg_5_95_std.pth 
 /media/titan/AI Research1/Data/CVPR2017/block_splits_by_image_all.pth 
 /media/titan/AI Research1/Data/CVPR2017/block_splits_by_image_single.pth


In [9]:
splits_all = torch.load(splits_all_path)
splits_single = torch.load(splits_single_path)

In [10]:
print(len(splits_all['splits']))
print(len(splits_all['splits'][0]))

print(len(splits_all['splits'][5]['train']))
print(len(splits_all['splits'][5]['val']))
print(len(splits_all['splits'][5]['test']))
print(splits_all['splits'][0]['train'][:40])
print(splits_all['splits'][1]['train'][:40])
print(splits_all['splits'][2]['train'][:10])
print(splits_all['splits'][3]['train'][:10])


6
3
7984
1996
1985
[0, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 27, 29, 33, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 48, 51, 52, 54, 55]
[1, 2, 3, 4, 6, 8, 9, 12, 13, 20, 25, 26, 27, 28, 30, 32, 33, 35, 37, 38, 39, 40, 44, 45, 46, 50, 52, 54, 56, 58, 59, 60, 62, 65, 68, 72, 73, 74, 76, 81]
[2, 3, 4, 5, 6, 7, 8, 10, 11, 13]
[1, 2, 4, 7, 9, 10, 12, 13, 14, 15]


In [11]:
# print(splits_single)
print(len(splits_single['splits'][0]['train']))
print(len(splits_single['splits'][0]['val']))
print(len(splits_single['splits'][0]['test']))

669
167
164


In [12]:
eeg_loaded = torch.load(eeg_dataset)

In [13]:
print(len(eeg_loaded))
print(eeg_loaded.keys())
dataset, labels, images = [eeg_loaded[k] for k in eeg_loaded.keys()]
print(len(labels))
print(len(images))
print(len(dataset))

print(labels)
print(images[0])
print(dataset[0]['eeg'].shape)
print(dataset[0])

3
dict_keys(['dataset', 'labels', 'images'])
40
1996
11965
['n02389026', 'n03888257', 'n03584829', 'n02607072', 'n03297495', 'n03063599', 'n03792782', 'n04086273', 'n02510455', 'n11939491', 'n02951358', 'n02281787', 'n02106662', 'n04120489', 'n03590841', 'n02992529', 'n03445777', 'n03180011', 'n02906734', 'n07873807', 'n03773504', 'n02492035', 'n03982430', 'n03709823', 'n03100240', 'n03376595', 'n03877472', 'n03775071', 'n03272010', 'n04069434', 'n03452741', 'n03792972', 'n07753592', 'n13054560', 'n03197337', 'n02504458', 'n02690373', 'n03272562', 'n04044716', 'n02124075']
n02951358_31190
torch.Size([128, 500])
{'eeg': tensor([[-0.0098,  0.0195,  0.0620,  ...,  0.0638,  0.0120, -0.0118],
        [-0.0045,  0.1303,  0.2673,  ...,  0.0894,  0.0342, -0.0082],
        [ 0.0215, -0.2017, -0.4305,  ..., -0.2022, -0.0940,  0.0188],
        ...,
        [ 0.0160,  0.0707,  0.1005,  ...,  0.2066,  0.1156,  0.0036],
        [-0.0046, -0.0084, -0.0119,  ...,  0.0007, -0.0026, -0.0053],
        [ 

In [14]:
opt = {
    # Dataset options
#     "iv": "image",
#     "offset": None,
    "results_file": "results.pkl",
    "subject": 0,
    "time_low": 20,
    "time_high": 460,
#     "run": "none",
    "eeg_dataset": eeg_dataset,
    "model_type": "model10",
    "splits_path": splits_all_path,
    "split_num": 0,
    "split_name": "train",
#     "fold": 5,
    #Training options
    "batch_size": 16,
    "optim": "Adam",
    "learning_rate": 0.001,
    "learning_rate_decay_by": 0.5,
    "learning_rate_decay_every": 10,
    "epochs": 100,
    "GPUindex": 0,
    "kind":"from-scratch",
    #Backend options
    "no_cuda": False,
    "classifier": None
}
opt = argparse.Namespace(**opt)
print(opt.time_low)

20


In [16]:
%load_ext autoreload
%autoreload 2
from torch.utils.data import DataLoader
# from data_loader import EEGDataset, Splitter, SplitterWithData
from data_loader_CVPR2017 import EEGDataset, Splitter
from EEG_Encoder.LSTM import classifier_LSTM
from EEG_Encoder.CNN import classifier_CNN
from EEG_Encoder.EEGNet import classifier_EEGNet
from EEG_Encoder.SyncNet import classifier_SyncNet
from EEG_Encoder.EEGChannelNet import classifier_EEGChannelNet
from EEG_Encoder.net_generator import Classifier
from EEG_Encoder.net_trainer import net_trainer
import matplotlib.pyplot as plt
from p_values import *
from torchinfo import summary

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
def load_dataset(
#              offset,
             eeg_dataset,
             splits_path,
             split_num, # (0-5) - 6 fold cross validation
             split_name,
#              total, 
#              classes,
#              classifier,
             batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             kind="from-scratch"):        
    # Load dataset
    dataset = EEGDataset(opt, eeg_dataset)
    print("DONE: LOAD DATASET")
#     # Create loaders for LSTM/MLP/CNN/SCNN/EEGNet/SyncNet/EEGChannelNet
#     if kind=="from-scratch":
#         relabel = False
#     if kind=="incremental":
#         relabel = False
#     if kind=="no-model-file":
#         relabel = True
    splitter = {split: Splitter(dataset,
                    splits_path,
                    split_num,
                    split_name = split) for split in ["train", "val", "test"]}
    loaders = {split: DataLoader(
                        splitter[split],
                        batch_size = batch_size,
                        drop_last = False,
                        shuffle = True)
                    for split in ["train", "val", "test"]}
    channel_idx = None    
    print("DONE: Create loaders for model")            
    return dataset, loaders, splitter

In [18]:
# Options
opt.classifier = "EEGNet"
opt.batch_size = 16
# opt.kind = "from-scratch"
# opt.run = "imagenet40-1000"
# opt.fold = 5

In [19]:
dataset, loaders, splitter = load_dataset(
#              offset,
             opt.eeg_dataset,
             opt.splits_path,
             opt.split_num, # (0-5) - 6 fold cross validation
             opt.split_name,
#              total, 
#              classes,
#              classifier,
             opt.batch_size,
#              GPUindex,
#              length, # 500
#              channel, # 128
#              min_CNN,
             opt,
             opt.kind)

DONE: LOAD DATASET
DONE: Create loaders for model


In [20]:
# loaders: divide the splits data in each fold with batch_size
# Each fold has {train: 8000 idx, val: 2000 idx, test: 2000 idx}
# Each loader batch has {train: 2000 idx, val: 250 idx, test: 250 idx}
print(type(dataset))
print(type(loaders))
print(len(loaders), [len(loaders[name]) for name in ["train", "val", "test"] ])
for i, (input, target) in enumerate(loaders["train"]):
    if i<20:
        print(f"{i+1}: Target size: {target.size()}; input size: {input.size()}")
for i in range(0, 40):
    eeg, label_val = splitter["val"][i]
    eeg, label_train = splitter["train"][i]
    print(f"{i+1}: Label val: {label_val}; label train: {label_train}")


<class 'data_loader_CVPR2017.EEGDataset'>
<class 'dict'>
3 [498, 125, 125]
1: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
2: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
3: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
4: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
5: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
6: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
7: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
8: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
9: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
10: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
11: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
12: Target size: torch.Size([16]); input size: torch.Size([16, 128, 440])
13: Target size: torch.Size([16]); input size:

In [21]:
net, nonclasses = Classifier(
                 n_classes,
                 classes,
                 opt.classifier,
                 opt.GPUindex,
                 length,
                 channel,
                 min_CNN,
                 opt.kind)
# print(len(nonclasses))
summary(net, input_size=(1,128, 440))

DONE: CREATE TORCH CLASSIFIER
classifier_EEGNet(
  (network): Sequential(
    (0): ZeroPad2d((110, 109, 0, 0))
    (1): Conv2d(1, 8, kernel_size=(1, 220), stride=(1, 1))
    (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(8, 16, kernel_size=(128, 1), stride=(1, 1), groups=8)
    (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ELU(alpha=1.0)
    (6): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
    (7): Dropout(p=0.5, inplace=False)
    (8): ZeroPad2d((8, 7, 0, 0))
    (9): Conv2d(16, 16, kernel_size=(1, 16), stride=(1, 1), groups=8)
    (10): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
    (11): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ELU(alpha=1.0)
    (13): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
    (14): Dropout(p=0.5, inplace=False)
  )
  (fc): Linear(in_features=208, out_features=40, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
classifier_EEGNet                        [1, 40]                   --
├─Sequential: 1-1                        [1, 16, 1, 13]            --
│    └─ZeroPad2d: 2-1                    [1, 1, 128, 659]          --
│    └─Conv2d: 2-2                       [1, 8, 128, 440]          1,768
│    └─BatchNorm2d: 2-3                  [1, 8, 128, 440]          16
│    └─Conv2d: 2-4                       [1, 16, 1, 440]           2,064
│    └─BatchNorm2d: 2-5                  [1, 16, 1, 440]           32
│    └─ELU: 2-6                          [1, 16, 1, 440]           --
│    └─AvgPool2d: 2-7                    [1, 16, 1, 110]           --
│    └─Dropout: 2-8                      [1, 16, 1, 110]           --
│    └─ZeroPad2d: 2-9                    [1, 16, 1, 125]           --
│    └─Conv2d: 2-10                      [1, 16, 1, 110]           528
│    └─Conv2d: 2-11                      [1, 16, 1, 110]           272
│    └─

In [23]:
model_path = (   opt.classifier+
                  "-"+
                  str(length)+
                  "-"+
                  str(channel) )
results_path = (  "results_"+ opt.classifier+
                  "-"+
                  str(length)+
                  "-"+
                  str(channel) )
print(model_path)
print(results_path)

EEGNet-440-128
results_EEGNet-440-128


In [24]:
print(opt)

Namespace(results_file='results.pkl', subject=0, time_low=20, time_high=460, eeg_dataset='/media/titan/AI Research1/Data/CVPR2017/eeg_5_95_std.pth', model_type='model10', splits_path='/media/titan/AI Research1/Data/CVPR2017/block_splits_by_image_all.pth', split_num=0, split_name='train', batch_size=16, optim='Adam', learning_rate=0.001, learning_rate_decay_by=0.5, learning_rate_decay_every=10, epochs=100, GPUindex=0, kind='from-scratch', no_cuda=False, classifier='EEGNet')


In [22]:
channel_idx=None
non_classes=None
if opt.kind=="from-scratch":
    loss_history, accuracy_val, accuracy_test = net_trainer(
            net,
            loaders,
            opt,
            channel_idx,
            nonclasses,
            None,
            True,
            model_path)

Epoch 1
Train Batch 100 (every 100 batch): Loss=3.7292; accuracy=0.0237
Train Batch 200 (every 100 batch): Loss=3.7793; accuracy=0.0300
Train Batch 300 (every 100 batch): Loss=3.7911; accuracy=0.0298
Train Batch 400 (every 100 batch): Loss=3.7075; accuracy=0.0298
Epoch 1 summary: train_loss: 3.6802 | train_acc: 0.0329 | val_loss: 3.5469 | val_acc: 0.0550
Epoch 2
Train Batch 100 (every 100 batch): Loss=3.3693; accuracy=0.0500
Train Batch 200 (every 100 batch): Loss=3.3983; accuracy=0.0572
Train Batch 300 (every 100 batch): Loss=3.3688; accuracy=0.0579
Train Batch 400 (every 100 batch): Loss=3.3170; accuracy=0.0589
Epoch 2 summary: train_loss: 3.3656 | train_acc: 0.0589 | val_loss: 3.1240 | val_acc: 0.0683
Epoch 3
Train Batch 100 (every 100 batch): Loss=3.0517; accuracy=0.0694
Train Batch 200 (every 100 batch): Loss=2.6072; accuracy=0.0791
Train Batch 300 (every 100 batch): Loss=2.4488; accuracy=0.0831
Train Batch 400 (every 100 batch): Loss=3.2280; accuracy=0.0853
Epoch 3 summary: train

Train Batch 100 (every 100 batch): Loss=2.2292; accuracy=0.2812
Train Batch 200 (every 100 batch): Loss=2.0922; accuracy=0.2850
Train Batch 300 (every 100 batch): Loss=1.8841; accuracy=0.2802
Train Batch 400 (every 100 batch): Loss=1.8261; accuracy=0.2767
Epoch 24 summary: train_loss: 1.9837 | train_acc: 0.2764 | val_loss: 1.7480 | val_acc: 0.3366
Epoch 25
Train Batch 100 (every 100 batch): Loss=1.8857; accuracy=0.2869
Train Batch 200 (every 100 batch): Loss=1.7647; accuracy=0.2794
Train Batch 300 (every 100 batch): Loss=2.0965; accuracy=0.2756
Train Batch 400 (every 100 batch): Loss=1.5872; accuracy=0.2767
Epoch 25 summary: train_loss: 1.9885 | train_acc: 0.2779 | val_loss: 1.7287 | val_acc: 0.3227
Epoch 26
Train Batch 100 (every 100 batch): Loss=1.9567; accuracy=0.2831
Train Batch 200 (every 100 batch): Loss=2.2340; accuracy=0.2800
Train Batch 300 (every 100 batch): Loss=2.2055; accuracy=0.2887
Train Batch 400 (every 100 batch): Loss=2.0771; accuracy=0.2884
Epoch 26 summary: train_lo

Train Batch 100 (every 100 batch): Loss=1.7897; accuracy=0.3525
Train Batch 200 (every 100 batch): Loss=1.4671; accuracy=0.3447
Train Batch 300 (every 100 batch): Loss=2.3199; accuracy=0.3369
Train Batch 400 (every 100 batch): Loss=1.2627; accuracy=0.3373
Epoch 47 summary: train_loss: 1.7795 | train_acc: 0.3359 | val_loss: 1.6322 | val_acc: 0.3776
Epoch 48
Train Batch 100 (every 100 batch): Loss=1.5585; accuracy=0.3612
Train Batch 200 (every 100 batch): Loss=1.3766; accuracy=0.3506
Train Batch 300 (every 100 batch): Loss=1.8104; accuracy=0.3494
Train Batch 400 (every 100 batch): Loss=2.0125; accuracy=0.3494
Epoch 48 summary: train_loss: 1.7500 | train_acc: 0.3514 | val_loss: 1.5504 | val_acc: 0.3952
Epoch 49
Train Batch 100 (every 100 batch): Loss=1.6720; accuracy=0.3269
Train Batch 200 (every 100 batch): Loss=1.4414; accuracy=0.3287
Train Batch 300 (every 100 batch): Loss=1.7870; accuracy=0.3263
Train Batch 400 (every 100 batch): Loss=2.0639; accuracy=0.3322
Epoch 49 summary: train_lo

Train Batch 100 (every 100 batch): Loss=1.4725; accuracy=0.3688
Train Batch 200 (every 100 batch): Loss=1.5420; accuracy=0.3762
Train Batch 300 (every 100 batch): Loss=2.0588; accuracy=0.3729
Train Batch 400 (every 100 batch): Loss=1.4678; accuracy=0.3762
Epoch 70 summary: train_loss: 1.6469 | train_acc: 0.3754 | val_loss: 1.4973 | val_acc: 0.4054
Epoch 71
Train Batch 100 (every 100 batch): Loss=1.4040; accuracy=0.3725
Train Batch 200 (every 100 batch): Loss=1.3142; accuracy=0.3606
Train Batch 300 (every 100 batch): Loss=1.7048; accuracy=0.3619
Train Batch 400 (every 100 batch): Loss=1.5003; accuracy=0.3658
Epoch 71 summary: train_loss: 1.6645 | train_acc: 0.3650 | val_loss: 1.4554 | val_acc: 0.4098
Epoch 72
Train Batch 100 (every 100 batch): Loss=1.7841; accuracy=0.4100
Train Batch 200 (every 100 batch): Loss=1.0375; accuracy=0.3994
Train Batch 300 (every 100 batch): Loss=2.2449; accuracy=0.3854
Train Batch 400 (every 100 batch): Loss=1.4918; accuracy=0.3781
Epoch 72 summary: train_lo

Train Batch 100 (every 100 batch): Loss=1.7920; accuracy=0.4069
Train Batch 200 (every 100 batch): Loss=1.2930; accuracy=0.4072
Train Batch 300 (every 100 batch): Loss=1.4310; accuracy=0.4038
Train Batch 400 (every 100 batch): Loss=1.2349; accuracy=0.3955
Epoch 93 summary: train_loss: 1.5801 | train_acc: 0.3962 | val_loss: 1.3982 | val_acc: 0.4346
Epoch 94
Train Batch 100 (every 100 batch): Loss=1.3569; accuracy=0.4131
Train Batch 200 (every 100 batch): Loss=1.0230; accuracy=0.4009
Train Batch 300 (every 100 batch): Loss=1.6697; accuracy=0.3910
Train Batch 400 (every 100 batch): Loss=1.8665; accuracy=0.3869
Epoch 94 summary: train_loss: 1.5942 | train_acc: 0.3878 | val_loss: 1.4894 | val_acc: 0.4044
Epoch 95
Train Batch 100 (every 100 batch): Loss=1.8913; accuracy=0.3919
Train Batch 200 (every 100 batch): Loss=1.1480; accuracy=0.3984
Train Batch 300 (every 100 batch): Loss=1.0906; accuracy=0.4079
Train Batch 400 (every 100 batch): Loss=1.7408; accuracy=0.4028
Epoch 95 summary: train_lo

ValueError: too many values to unpack (expected 3)

In [26]:
torch.save(results, results_path+'.obj')

NameError: name 'loss_history' is not defined

In [ ]:
loaded_results = torch.load(results_path+'.obj')
print(loaded_results)